In [1]:
import networkx as nx

# Extract the Network from amazon-meta.txt

In [2]:
# Extract the data from the file
file = "./amazon-meta.txt"
with open(file, encoding = 'utf8') as f:
    data = f.readlines()

# Create a network using this data
G = nx.Graph()
i = 0 
for line in data:
    currentLine = line.split() #Split the line into a list where each word is a list item 
    if len(currentLine) >= 2 and currentLine[0] == "Id:":
        G.add_node(currentLine[1])

        
        

print(nx.info(G))

    
    

Name: 
Type: Graph
Number of nodes: 548552
Number of edges: 0
Average degree:   0.0000


# Extract the network communities and analysis of their homogeneity with respect to product categories